# Open CSV file 📑

In [11]:

import pandas as pd
lab = pd.read_csv('/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/CSV/Annotation_boxes_Factory_plate_set1-3.csv')
lab = lab.drop(columns=['Unnamed: 0'])
lab = lab[lab['Split'] == 'Train']
print(f'dataset Lab lab: {lab.shape}')
lab.head()

dataset Lab lab: (558, 13)


,image_id,width,height,file_name,area,bbox,Split,set,xmin,ymin,xmax,ymax,img_path
0,1,1920,1080,ABS1_48.jpg,583617.1232,"[697.96, 78.48, 668.32, 873.26]",Train,set1,697.96,78.48,1366.28,951.74,/media/HDD/PaperPlateQC-Dataset/Factory-plate-...
1,2,1920,1080,ABS1_62.jpg,631050.8628,"[653.41, 130.16, 703.96, 896.43]",Train,set1,653.41,130.16,1357.37,1026.59,/media/HDD/PaperPlateQC-Dataset/Factory-plate-...
2,3,1920,1080,ABS1_83.jpg,613922.8062,"[653.41, 89.17, 680.79, 901.78]",Train,set1,653.41,89.17,1334.20,990.95,/media/HDD/PaperPlateQC-Dataset/Factory-plate-...
3,4,1920,1080,AFS1_43.jpg,694583.6612,"[648.06, 46.4, 727.13, 955.24]",Train,set1,648.06,46.40,1375.19,1001.64,/media/HDD/PaperPlateQC-Dataset/Factory-plate-...
4,5,1920,1080,AFS1_66.jpg,698228.5788,"[637.37, 49.96, 737.82, 946.34]",Train,set1,637.37,49.96,1375.19,996.30,/media/HDD/PaperPlateQC-Dataset/Factory-plate-...


#  ย้ายรูปไปรวมที่เดียวกัน 🗂

In [7]:
# save_path = '/media/HDD/dataset_cctv_plate/data_defect/defect_train_20_Mar_25_set1_2/'

# Create directories for images and labels 📥

In [12]:
import pandas as pd
import os
import ast
import shutil
from PIL import Image

# Create directories for images and labels
base_path = '/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2'  # Change this to your desired base path
os.makedirs(os.path.join(base_path, 'images', 'train'), exist_ok=True)
os.makedirs(os.path.join(base_path, 'images', 'val'), exist_ok=True)
os.makedirs(os.path.join(base_path, 'labels', 'train'), exist_ok=True)
os.makedirs(os.path.join(base_path, 'labels', 'val'), exist_ok=True)

# ➖ Train set

In [17]:
def convert_to_yolo_format(xmin, ymin, xmax, ymax, width, height):
    x_center = ((xmin + xmax) / 2) / width
    y_center = ((ymin + ymax) / 2) / height
    box_width = (xmax - xmin) / width
    box_height = (ymax - ymin) / height
    return [x_center, y_center, box_width, box_height]

In [18]:
file_name_list = list(set(lab['file_name'])) 

for no in range(len(file_name_list)):
    df = lab[lab['file_name'] == file_name_list[no]]

    image_width = df['width'].tolist()[0]
    image_height = df['height'].tolist()[0]
    cate = df['set'].tolist()[0]
    original_path = df['img_path'].tolist()[0]

    # ✂️ train labels
    train_labels = f'{base_path}/labels/train/' 
    train_images = f'{base_path}/images/train/' 

    # สร้างไดกชันนารีเพื่อจัดเก็บไฟล์ป้ายกำกับตามชื่อภาพ
    labels = {}
    for index, row in df.iterrows():
        label_str = f"0 {convert_to_yolo_format(row['xmin'], row['ymin'], row['xmax'], row['ymax'], image_width, image_height)[0]} {convert_to_yolo_format(row['xmin'], row['ymin'], row['xmax'], row['ymax'], image_width, image_height)[1]} {convert_to_yolo_format(row['xmin'], row['ymin'], row['xmax'], row['ymax'], image_width, image_height)[2]} {convert_to_yolo_format(row['xmin'], row['ymin'], row['xmax'], row['ymax'], image_width, image_height)[3]}"
        if row['file_name'] in labels:
            labels[row['file_name']].append(label_str)
        else:
            labels[row['file_name']] = [label_str]

    # บันทึกข้อมูลใหม่เป็นไฟล์ .txt สำหรับ YOLO
    for file_name, label_list in labels.items():
        with open(f"{train_labels}{cate}_{file_name.replace('.jpg', '.txt')}", 'w') as file:
            for label in label_list:
                file.write(f"{label}\n")
        # save iamges
        new_filename = f"{train_images}{cate}_{file_name}"
        shutil.copy2(original_path, new_filename)
       
        print(f"{train_labels}{cate}_{file_name.replace('.jpg', '.txt')} saved")
        print(f"Image processed and saved to {new_filename}")

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_NFS2_3845.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set2_NFS2_3845.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_GBS1_1603.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set1_GBS1_1603.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_DBS1_1859.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set1_DBS1_1859.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_BBS1_3611.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_BFS2_316.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set2_BFS2_316.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_GFS2_1601.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set2_GFS2_1601.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_CFS1_464.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set1_CFS1_464.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_GFS1_3167.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_BBS2_804.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set2_BBS2_804.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_IBS2_3488.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set2_IBS2_3488.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_GBS1_3098.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set1_GBS1_3098.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_EFS2_2805.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-B

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_ABS2_69.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set2_ABS2_69.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_BBS1_739.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set1_BBS1_739.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_KBS2_3535.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set2_KBS2_3535.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_CFS1_2150.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/t

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_NBS1_3769.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set1_NBS1_3769.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_FBS1_2887.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set1_FBS1_2887.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_MBS1_1390.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set1_MBS1_1390.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_MFS1_1343.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_FFS2_2917.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set2_FFS2_2917.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_CFS2_2287.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set2_CFS2_2287.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_FFS2_2911.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set2_FFS2_2911.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_CBS2_2467.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_MFS1_1435.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set1_MFS1_1435.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_IBS1_3502.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set1_IBS1_3502.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_CBS2_2253.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set2_CBS2_2253.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_CBS1_439.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_DBS1_505.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set1_DBS1_505.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_JFS1_3308.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set1_JFS1_3308.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_BFS2_279.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set2_BFS2_279.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_NFS2_3809.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set1_MBS1_2697.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set1_MBS1_2697.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_NFS2_3840.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set2_NFS2_3840.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_GFS2_1537.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train/set2_GFS2_1537.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/train/set2_JFS2_2557.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO

# ➖ Validation set

In [19]:
def convert_to_yolo_format(xmin, ymin, xmax, ymax, width, height):
    x_center = ((xmin + xmax) / 2) / width
    y_center = ((ymin + ymax) / 2) / height
    box_width = (xmax - xmin) / width
    box_height = (ymax - ymin) / height
    return [x_center, y_center, box_width, box_height]

In [22]:
file_name_list = list(set(lab['file_name'])) 

for no in range(len(file_name_list)):
    df = lab[lab['file_name'] == file_name_list[no]]

    image_width = df['width'].tolist()[0]
    image_height = df['height'].tolist()[0]
    cate = df['set'].tolist()[0]
    original_path = df['img_path'].tolist()[0]

    # ✂️ train labels
    train_labels = f'{base_path}/labels/val/' 
    train_images = f'{base_path}/images/val/' 

    # สร้างไดกชันนารีเพื่อจัดเก็บไฟล์ป้ายกำกับตามชื่อภาพ
    labels = {}
    for index, row in df.iterrows():
        label_str = f"0 {convert_to_yolo_format(row['xmin'], row['ymin'], row['xmax'], row['ymax'], image_width, image_height)[0]} {convert_to_yolo_format(row['xmin'], row['ymin'], row['xmax'], row['ymax'], image_width, image_height)[1]} {convert_to_yolo_format(row['xmin'], row['ymin'], row['xmax'], row['ymax'], image_width, image_height)[2]} {convert_to_yolo_format(row['xmin'], row['ymin'], row['xmax'], row['ymax'], image_width, image_height)[3]}"
        if row['file_name'] in labels:
            labels[row['file_name']].append(label_str)
        else:
            labels[row['file_name']] = [label_str]

    # บันทึกข้อมูลใหม่เป็นไฟล์ .txt สำหรับ YOLO
    for file_name, label_list in labels.items():
        with open(f"{train_labels}{cate}_{file_name.replace('.jpg', '.txt')}", 'w') as file:
            for label in label_list:
                file.write(f"{label}\n")
        # save iamges
        new_filename = f"{train_images}{cate}_{file_name}"
        shutil.copy2(original_path, new_filename)
       
        print(f"{train_labels}{cate}_{file_name.replace('.jpg', '.txt')} saved")
        print(f"Image processed and saved to {new_filename}")

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_NFS2_3845.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set2_NFS2_3845.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_GBS1_1603.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set1_GBS1_1603.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_DBS1_1859.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set1_DBS1_1859.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_BBS1_3611.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_BBS1_186.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set1_BBS1_186.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_BFS1_3636.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set1_BFS1_3636.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_CBS1_2423.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set1_CBS1_2423.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_BFS1_3615.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Ma

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_CBS2_2452.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set2_CBS2_2452.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_GFS1_3108.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set1_GFS1_3108.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_CFS1_2443.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set1_CFS1_2443.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_BBS2_3633.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_MBS2_1416.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set2_MBS2_1416.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_BBS1_3604.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set1_BBS1_3604.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_GBS1_1662.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set1_GBS1_1662.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_GBS2_1546.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_GBS2_3004.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set2_GBS2_3004.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_GBS2_3196.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set2_GBS2_3196.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_NBS2_3825.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set2_NBS2_3825.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_BFS1_218.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_M

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_JBS1_2576.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set1_JBS1_2576.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_BBS1_261.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set1_BBS1_261.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_NBS1_3706.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set1_NBS1_3706.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_NFS1_3721.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Ma

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_BFS2_707.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set2_BFS2_707.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_GBS2_3183.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set2_GBS2_3183.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_NFS2_3733.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set2_NFS2_3733.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_JFS1_2574.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Ma

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_CFS2_2256.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set2_CFS2_2256.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_MBS2_2662.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set2_MBS2_2662.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_DBS2_1939.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set2_DBS2_1939.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_GFS2_3081.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_

/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_GBS2_3203.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set2_GBS2_3203.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_FBS2_2894.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set2_FBS2_2894.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set1_MFS1_2610.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val/set1_MFS1_2610.jpg
/media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/labels/val/set2_MBS2_2640.txt saved
Image processed and saved to /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_

## Manage dataset by split 80:20

In [23]:
# [1] - Randomly select 20% images of all

val_path = f'{base_path}/labels/val/'  
import os
os.chdir(val_path)

import glob
val_path_all = glob.glob('*')
print(len(val_path_all))

n = int(len(val_path_all)*0.2) # 20 % of train set

import random  
# Randomly select a few images  
selected_val_path = random.sample(val_path_all, n) 
print(len(selected_val_path))

selected_val_path_remove = list(set(val_path_all )-set(selected_val_path))
print(len(selected_val_path_remove))

558
111
447


In [24]:
# [2] - delete from validation set


val_path = f'{base_path}/labels/val/' 
import os
os.chdir(val_path)

for i in selected_val_path_remove:
    !rm {i}

val_path = f'{base_path}/images/val/' 
import os
os.chdir(val_path)

a_ =[]
for i in selected_val_path_remove:
    a = i.split('.txt')[0]+'.jpg'
    a_.append(a)
    
for i in a_:
    !rm {i}

In [25]:
# [3] - delete from train set

train_path = f'{base_path}/labels/train/' 
import os
os.chdir(train_path)


for i in selected_val_path:
    !rm {i}

train_path = f'{base_path}/images/train/' 
import os
os.chdir(train_path)


a_ =[]
for i in selected_val_path:
    a = i.split('.txt')[0]+'.jpg'
    a_.append(a)

for i in a_:
    !rm {i}

## Create .yaml

In [26]:
# Define the paths and class information
data_yaml_path = '/home/yupaporn/code/Defect-Detection-2024/yolov5/Plate_Box_train_20_Mar_25_set1_2.yaml'  # Update with your desired path

# Number of classes and their names
num_classes = 1  # Update with the actual number of classes
class_names = ['Plate-Box']  # Replace with actual class names

# Write to the YAML file
with open(data_yaml_path, 'w') as file:
    file.write(f"train: /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/train\n")
    file.write(f"val: /media/HDD/PaperPlateQC-Dataset/Factory-plate-data/YOLO-Box/train_20_Mar_25_set1_2/images/val\n")
    file.write(f"\nnc: {num_classes}\n")
    file.write(f"names: {class_names}\n")

print("data.yaml file created successfully!")

data.yaml file created successfully!
